In [1]:
import numpy as np
import pandas as pd
# import streamlit as st
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sql = """
SELECT a.retailer_id
      ,a.order_date
      ,a.order_in_day
      ,a.previous_order_date
      ,a.day_diff
      ,b.current_kship_stage_detail
FROM `kiotvietplus.kv_shipping_dwh.vw_kship_bill_lading_facts_aggregate` a
join `kiotvietplus-dev.Data_lead_opt_for_kship.scoring_kship_merchant_table`  b
on a.retailer_id = b.retailer_id
and b.run_date = '2022-12-28'
"""

In [4]:
df_kship = pd.read_csv('./bq-results-20221229-073152-1672299149657.csv')

In [5]:
df_kship.shape

(1539733, 6)

In [6]:
df_kship.head()

,retailer_id,order_date,order_in_day,previous_order_date,day_diff,current_kship_stage_detail
0,102754,2021-03-15,45,2021-03-13,2,Active
1,102754,2021-06-11,22,2021-06-10,1,Active
2,102754,2022-09-05,39,2022-09-03,2,Active
3,102754,2022-09-19,28,2022-09-17,2,Active
4,102754,2022-10-29,31,2022-10-28,1,Active


In [10]:
df_kship = df_kship[df_kship.order_date >= '2022-01-01']

In [11]:
df_kship.shape

(762288, 6)

In [23]:
def Calculate_BPD(df,filer_field_id,retailer_id,date_diff_calculated_field,percentile_scale=90):
    """
    # Compute the BPD xxx th percentile
    exp: df_kship['BPD_90_Percent'] = df_kship.apply(lambda x: (Calculate_BPD(df_kship,'retailer_id',x.retailer_id,'day_diff',percentile_scale=90)), axis=1)
    """
    ecdf = ECDF(df[df[filer_field_id]==retailer_id][date_diff_calculated_field].dropna())
    BPD_Days = np.percentile(ecdf.x,percentile_scale)
    return BPD_Days



def Calculate_risk_forChurn(df,filer_field_id,retailer_id,order_date_field,BPD_percent_field):
    """
    # function takes customer ID and returns if at risk to churn
    exp:
    """
    Current_Date = pd.to_datetime(df[order_date_field].max())
    retailer_last_orderdate = pd.to_datetime(df[df[filer_field_id] == retailer_id][order_date_field].max())
    retailer_last_orderdate_from_CurrentDate = Current_Date - retailer_last_orderdate

    BPD_percent = df[df[filer_field_id] == retailer_id][BPD_percent_field].head(1)
    # BPD_percent = BPD_percent.dt.days
    BPD_percent = pd.DataFrame(BPD_percent)
    BPD_percent = BPD_percent[BPD_percent_field].values

    if (retailer_last_orderdate_from_CurrentDate.days >= BPD_percent[0]):
        churnrisk = 'Customer is at risk to churn'
    else:
         churnrisk = 'Customer is not at risk to churn'
        # select customer id
    return churnrisk

In [13]:
df_kship['BPD_90_Percent'] = df_kship.apply(lambda x: (Calculate_BPD(df_kship,'retailer_id',x.retailer_id,'day_diff',percentile_scale=90)), axis=1)


Need to modify this code:

1. Only run ECDF for merchant who has more than 5 or 10 day orders

2. If merchant has less than 5 or 10 days order -> use rule base


In [14]:
df_kship.head()

,retailer_id,order_date,order_in_day,previous_order_date,day_diff,current_kship_stage_detail,BPD_90_Percent
2,102754,2022-09-05,39,2022-09-03,2,Active,2.0
3,102754,2022-09-19,28,2022-09-17,2,Active,2.0
4,102754,2022-10-29,31,2022-10-28,1,Active,2.0
5,112946,2022-07-11,32,2022-07-10,1,Active,1.0
6,112946,2022-11-29,23,2022-11-28,1,Active,1.0


In [24]:
## Chi can lay ngay giao dịch max cua tung KH roi moi du bao churn -> cai thien performance + dung ban chat hon
df_kship['churnrisk'] = df_kship.apply(lambda x: (Calculate_risk_forChurn(df_kship,'retailer_id',x.retailer_id,'order_date','BPD_90_Percent')), axis=1)


In [25]:
df_kship.head()

,retailer_id,order_date,order_in_day,previous_order_date,day_diff,current_kship_stage_detail,BPD_90_Percent,churnrisk
2,102754,2022-09-05,39,2022-09-03,2,Active,2.0,Customer is not at risk to churn
3,102754,2022-09-19,28,2022-09-17,2,Active,2.0,Customer is not at risk to churn
4,102754,2022-10-29,31,2022-10-28,1,Active,2.0,Customer is not at risk to churn
5,112946,2022-07-11,32,2022-07-10,1,Active,1.0,Customer is not at risk to churn
6,112946,2022-11-29,23,2022-11-28,1,Active,1.0,Customer is not at risk to churn


In [26]:
df_kship.to_csv('churn_result.csv')

In [27]:
df_kship = df_kship.drop(columns='churnrisk',axis=1)

In [29]:
df_kship['order_date'] = pd.to_datetime(df_kship['order_date'])

In [30]:
df_kship_max = df_kship.loc[df_kship.groupby(['retailer_id'])['order_date'].idxmax()]

In [34]:
df_kship_max['churnrisk'] = df_kship_max.apply(lambda x: (Calculate_risk_forChurn(df_kship_max,'retailer_id',x.retailer_id,'order_date','BPD_90_Percent')), axis=1)

In [36]:
df_kship_max.to_excel('churn_result_2.xlsx')